In [27]:
%load_ext autoreload
%autoreload 1
%aimport src.my, src.net, src.data, src.models, src.text_utils, src.huse, src.losses

import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle

# import net, data
import src.huse as huse
import src.models as ms
import src.net as net
import src.text_utils as tu
import src.my as my
from src.my import p
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 45)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

dir_data = 'data/'
dir_out = 'out/'
os.makedirs(dir_out, exist_ok=True)

SEED = 34
N_CPU = os.cpu_count()

np.random.seed(SEED)
rng = np.random.default_rng(SEED)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
Xy = pd.read_parquet(dir_out+'prepared_df.pq')
X_test = pd.read_parquet(dir_out+'prepared_test.pq')
Xy[:2]

,product_id,category_id,shop_id,category_name,fold,text
0,325286,251,493,электроника смартфоны телефоны аксессуары...,4,зарядный кабель borofone bx1 lightning ай...
1,888134,748,6081,одежда женская одежда белье купальники трусы,3,трусы sela трусы слипы эластичного бесшов...


In [3]:
X_test[:2]

,product_id,shop_id,text
0,1997646,6217,светодиодная лента smart led strip light ...
1,927375,1796,стекло пленка керамик матовое honor lite ...


In [4]:
CHECKPOINT_DIR = dir_out + 'huse_model'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_DIR

'out/huse_model'

### Построение adjacency matrix Aij семантического графа
Эта матрица нужна для вычисления семантической лосс-функции
Элемент матрицы Aij показывает, какое расстояние между эмбеддингом названия i категории и j категории

In [5]:
cat_df = Xy[['category_id','category_name']].drop_duplicates(subset=['category_id']).sort_values('category_id').reset_index(drop=True).copy()
cat_df

,category_id,category_name
0,0,одежда женская одежда белье купальники ку...
1,1,одежда женская одежда белье купальники пл...
2,2,одежда женская одежда белье купальники ма...
3,3,одежда женская одежда белье купальники ко...
4,4,одежда детская одежда одежда мальчиков но...
...,...,...
869,869,товары дома текстиль подушки подушки наво...
870,870,электроника умный дом безопасность видеон...
871,871,товары дома декор интерьер свечи подсвечн...
872,872,товары дома декор интерьер картины панно ...


Всего 874 категории, название категории максимум 14 слов

In [6]:
cat_df['category_name'].apply(tu.number_words).describe()

count   874.000
mean      7.283
std       1.737
min       3.000
25%       6.000
50%       7.000
75%       8.000
max      14.000
Name: category_name, dtype: float64

In [7]:
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoConfig

Матрица будет размером 874 на 874

In [8]:
N_classes = len(cat_df)
Aij = torch.zeros((N_classes, N_classes), dtype=torch.float32)
N_classes, Aij.shape

(874, torch.Size([874, 874]))

Получим эмбеддинги названий категорий с помощью берта

In [9]:
class CategoryNameDset(Dataset):
    def __init__(self, df:pd.DataFrame, tokenizer, max_length:int=14):
        self.inputs = tokenizer(df['category_name'].to_list(), padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

    def __len__(self): return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        item = {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            }
        if "token_type_ids" in self.inputs:
            item["token_type_ids"] = self.inputs["token_type_ids"][idx]
        return item


class CategoryNameModel(nn.Module):
    def __init__(self, model_name:str) -> None:
        super().__init__()

        self.model = AutoModel.from_pretrained(model_name)
        self.pool = ms.MeanPooling()

    def forward(self, inputs):
        inputs = {k:v for k,v in inputs.items() if k != 'label'}
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = CategoryNameModel("cointegrated/LaBSE-en-ru").to(net.device)
_ = model.eval()

In [11]:
cat_ds = CategoryNameDset(cat_df, tokenizer)
cat_dl = DataLoader(cat_ds, batch_size=128, num_workers=1, shuffle=False)

In [12]:
len(cat_ds), len(cat_dl)

(874, 7)

In [13]:
class_embs = []
with torch.inference_mode():
    for batch in tqdm(cat_dl):
        batch = {k: v.to(net.device) if hasattr(v, 'to') else v for k, v in batch.items()}
        out = model(batch)
        class_embs.append(out)
class_embs = torch.cat(class_embs)
class_embs.shape

  0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([874, 768])

Посчитаем косинусные расстояния между каждым эмбеддингом и запишем их в матрицу

In [14]:
for iy in tqdm(range(N_classes)):
    for ix in range(iy, N_classes):
        # cosine distance
        d = 1 - F.cosine_similarity(class_embs[iy].reshape(1,-1), class_embs[ix].reshape(1,-1))
        Aij[ix][iy] = Aij[iy][ix] = d

  0%|          | 0/874 [00:00<?, ?it/s]

In [17]:
Aij = Aij.to(net.device)

Передадим Aij в модель HUSE для подсчета семантической фуннкции потерь. Использовал разные learning rates для text и image towers

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar, ModelCheckpoint
import src.net as net, src.data as data
from pytorch_lightning.plugins.precision import MixedPrecisionPlugin

torch.use_deterministic_algorithms(False)
torch.set_float32_matmul_precision('medium')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class CFG:
    lr = 5e-4
    bert_lr = 5e-5

    model_name = 'cointegrated/LaBSE-en-ru'
    scheduler='cosine'
    num_cycles=0.5
    num_warmup_steps=30
    epochs=30
    batch_size=196
    max_length = 64
    gradient_checkpointing=False
    gradient_accumulation_steps=1
    max_grad_norm=50
    precision = 16

    emb_dim = 256
    shop_emb_dim = 10
    num_classes = N_classes

    # params big loss
    weight_class_L = 0.8
    weight_semantic_L = 10
    weight_gap_L = 0.1
    margin_semantic = 0.7

def train_fold(Xy:pd.DataFrame, Xy_test:pd.DataFrame, fold:int=0):
    print('[TRAIN FOLD]:',fold)
    net.set_seed(SEED + 10*fold)
    NAME_CKPT = f'best_f{fold}'

    dm = data.HuseDataModule(Xy, Xy_test, fold=fold, batch_size = CFG.batch_size, val_bs=196, n_cpu=10, cfg=CFG)

    CFG.num_train_steps = int(dm.len_train/CFG.batch_size*CFG.epochs)

    model = huse.HUSEModule(Aij, CFG)

    tq = TQDMProgressBar(refresh_rate=5)

    es = EarlyStopping('val_f1', min_delta=0.001,patience=5,verbose=True, mode='max', check_on_train_epoch_end=False)

    chpt = ModelCheckpoint(dirpath=CHECKPOINT_DIR,filename=f'best_f{fold}',  monitor='val_f1',mode='max')

    trainer = pl.Trainer(
        precision=CFG.precision,
        plugins=[MixedPrecisionPlugin(precision=16,device='cuda')],
    callbacks=[tq,es,chpt],
    max_epochs=CFG.epochs,
    # deterministic = True,
    accelerator='auto',
    accumulate_grad_batches = CFG.gradient_accumulation_steps,
    gradient_clip_val = CFG.max_grad_norm,
    # val_check_interval = 0.001,
#     logger = False,
    log_every_n_steps = 50,
    enable_model_summary = True if fold==0 else False)

    trainer.fit(model, datamodule=dm)

    del trainer
    torch.cuda.empty_cache()
    gc.collect()

    return chpt.best_model_score.cpu().item()

res = []
for fold in sorted(Xy['fold'].unique()):
    res_fold = train_fold(Xy, X_test, fold=fold)
    res.append((fold,res_fold))
    break

в итоге получили f1 ~ 0.88